In [ ]:
import sys
sys.path.append("../")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
#import pandas as pd
import numpy as np
import cv2
#from PIL import Image
import random
#import tensorflow as tf
#import re
import datetime
import io
#from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import pickle
#import string
#from keras.models import Model, load_model
#from keras.layers import Input, LSTM, Dense, Conv2D, MaxPooling2D, Reshape, Dropout, BatchNormalization, Activation
#from utils import score_prediction, generate_token_index, y_labels
import yaml 
import json
from preprocessing.preproc_functions import read_image, read_query, normalize_0_mean_1_variance, normalize_0_1, read_annotation
from predictors.predictor import PredictorFCN
from data_generators.data_generator import DataGenerator
#from utils.score_prediction import pixel_accuracy, mean_accuracy, mean_IoU, freq_weighted_mean_IoU, score_prediction
from tensorflow.keras.models import load_model, model_from_json

%matplotlib inline

In [ ]:
with open('../configs/config.yml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
config['labels_file'] = '../' + config['labels_file']
config['dataset_folder'] = '../' + config['dataset_folder']

config['network']['graph_path'] = '../' + config['network']['graph_path']
config['predict']['weights_file'] = '../' + config['predict']['weights_file']

y_size = config['image']['image_size']['y_size']
x_size = config['image']['image_size']['x_size']
y_query_size = config['image']['query_image_size']['y_size']
x_query_size = config['image']['query_image_size']['x_size']

In [ ]:
with open(config['labels_file']) as f:
    dataset = json.load(f)

### Load model

In [ ]:
json_file = open(config['network']['graph_path'], 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights(config['predict']['weights_file'])

### Predict

In [ ]:
items = dataset['test']

In [ ]:
index = random.randint(0, len(items)-1)
elem = items[index]

query = read_query(config['dataset_folder'], elem['query'], y_query_size, x_query_size, black_white = False)
image = read_image(config['dataset_folder'], elem['image']['filename'], y_size, x_size, black_white = False)
annotation = read_annotation(config['dataset_folder'], elem['image']['annotation'], y_size, x_size)

query = np.expand_dims(query, axis=0)
image = np.expand_dims(image, axis=0)

In [ ]:
output_raw = model.predict([query, image])
output_raw = output_raw/np.max(output_raw)
output = np.where(output_raw > 0.5, 1., 0.)

### Plot

In [ ]:
#index = random.randint(0, len(query)-1)
fig = plt.figure(figsize=(15, 15))
plt.subplot(1, 4, 1)
plt.imshow(query[0, ...]/255.)
plt.subplot(1, 4, 2)
plt.imshow(image[0, ...])
plt.subplot(1, 4, 3)
plt.imshow(annotation)
plt.subplot(1, 4, 4)
plt.imshow(output[0, ...])